In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# !pip install torchvision
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# TODO: 讀取數據集，'train-images.pt' 和 'train-labels.csv'
images_raw = torch.load('/kaggle/input/moai-2025-training/train_images.pt', weights_only=False)
labels_raw = pd.read_csv('/kaggle/input/moai-2025-training/train_labels.csv')

# TODO: 歸一化數據集並轉換為 torch.Tensor
images = images_raw/255.0
labels = torch.tensor(labels_raw['label'].values)


dataset = TensorDataset(images, labels)

In [3]:
# TODO: 創建數據集，並按照 8:2 劃分成訓練集和驗證集

train_dataset, val_dataset = random_split(dataset, [48000, 12000])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

In [4]:
for data, targets in train_loader:
    print("Data shape:", data.shape)
    print("Targets shape:", targets.shape)
    break

Data shape: torch.Size([128, 28, 28])
Targets shape: torch.Size([128])


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
from torchvision import datasets, models, transforms
model = models.resnet50(pretrained=True).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 147MB/s] 


In [12]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Sequential(
    nn.Linear(2048, 128, bias = True),
    nn.ReLU(inplace = True),
    nn.Linear(128, 10, bias = True)
)

In [13]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():  # Last residual block
    param.requires_grad = True

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [17]:
import torch
from tqdm import tqdm

num_epochs = 20

for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    model.train()
    for data, targets in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        data = data.unsqueeze(1)
        data, targets = data.to(device), targets.to(device)
        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for data, targets in tqdm(val_loader, desc="Validation"):
            data = data.unsqueeze(1)
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, preds = outputs.max(1)
            val_correct += (preds == targets).sum().item()
            val_total += targets.size(0)

    val_acc = val_correct / val_total
    print(f"Validation Accuracy: {val_acc * 100:.2f}%\n")

Epoch [1/20]


Validation:   0%|          | 0/94 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 1, 7, 7], expected input[1, 128, 28, 28] to have 1 channels, but got 128 channels instead